In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)
import json

In [2]:
df_work_raw = catalog.load('raw/openalex/work_dev#parquet')

[02/01/25 19:40:25] INFO     Loading data from raw/openalex/work_dev#parquet                    ]8;id=433832;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=750450;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

# Profiling

In [3]:
df_work_raw.shape

(1000, 51)

In [4]:
df_work_raw.head(3)

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,abstract_inverted_index_v3,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'doi': 'https://doi.org/10.1016/s0378-4371(98...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': False, 'is_oa'...",[{'affiliations': [{'institution_ids': array([...,[],3,6,[https://openalex.org/A5051451815],[https://openalex.org/I4210125245],"{'currency': 'USD', 'provenance': 'doaj', 'val...",None,30.137,True,ngrams,1339,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '534', 'issue': '3-4', 'last_pa...",False,False,{'display_name': 'Statistical Mechanics and En...,[{'display_name': 'Statistical Mechanics and E...,[{'display_name': 'Hamiltonian (control theory...,"[{'display_name': 'Harmonic oscillator', 'id':...",[],1,"[{'is_accepted': False, 'is_oa': False, 'is_pu...",None,[{'display_name': 'Affordable and clean energy...,[],[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...",None,None,https://api.openalex.org/works?filter=cites:W2...,"[{'cited_by_count': 49, 'year': 2024}, {'cited...",2025-01-22T01:09:23.293089,2016-06-24
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'doi': 'https://doi.org/10.1016/j.envint.2020...,en,"{'is_accepted': True, 'is_oa': True, 'is_publi...",review,journal-article,"[crossref, doaj, pubmed]","{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],1,5,[https://openalex.org/A5069680875],"[https://openalex.org/I151201029, https://open...","{'currency': 'USD', 'provenance': 'doaj', 'val...","{'currency': 'USD', 'provenance': 'doaj', 'val...",8.630,True,pdf,525,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '105876', 'issue': None, 'last_...",False,False,{'display_name': 'Air Quality and Health Impac...,[{'display_name': 'Air Quality and Health Impa...,"[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'descriptor_name': 'Air Pollutants', 'descri...",4,"[{'is_accepted': True, 'is_oa': True, 'is_publ...","{'is_accepted': True, 'is_oa': True, 'is_publi...",[{'display_name': 'Good health and well-being'...,"[{'award_id': None, 'funder': 'https://openale...",[],[],85,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W4399338999, https://ope...","{'""Best': None, '""best': None, '#876.': None, ...",None,https://api.openalex.org/works?filter=cites:W3...,"[{'cited_by_count': 170, 'year': 2024}, {'cite...",2025-01-25T03:47:29.633744,2020-07-02
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'doi': 'https://doi.org/10.1016/s0023-6438(03...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_ha

In [5]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw.dtypes


id                                string[python]
doi                               string[python]
title                             string[python]
display_name                      string[python]
publication_year                           Int64
publication_date                  string[python]
ids                                       object
language                          string[python]
primary_location                          object
type                              string[python]
type_crossref                     string[python]
indexed_in                                object
open_access                               object
authorships                               object
institution_assertions                    object
countries_distinct_count                   Int64
institutions_distinct_count                Int64
corresponding_author_ids                  object
corresponding_institution_ids             object
apc_list                                  object
apc_paid           

# Nodo

In [6]:
def land_work2authorships_openalex(df_work_raw):
    # Seleccionar las columnas necesarias y convertir los tipos de datos
    df_work2authorships = df_work_raw[['id', 'authorships']].convert_dtypes()
    df_work2authorships.rename(columns={"id": "work_id"}, inplace=True)

    # Expandir la lista de authorships
    df_work2authorships_exploded = df_work2authorships.explode('authorships', ignore_index=True)

    # Normalizar la información de authorships
    df_authorships_norm = pd.json_normalize(df_work2authorships_exploded['authorships'])
    df_authorships_norm.rename(columns={"author.id": "author_id"}, inplace=True)

    # Combinar work_id con la información normalizada de authorships
    df_work2authorships = df_work2authorships_exploded[['work_id']].join(df_authorships_norm)

    # Extraer la relación work-author
    df_work2author = df_work2authorships[['work_id', 'author_id', 'author_position']]

    # Expandir la lista de instituciones asociadas a cada autor
    df_author2institution_exploded = df_work2authorships.explode('institutions', ignore_index=True)

    # Normalizar la información de instituciones
    df_institution_norm = pd.json_normalize(df_author2institution_exploded['institutions'])
    df_institution_norm.drop(columns=['lineage'], errors='ignore', inplace=True)

    # Combinar author_id con la información normalizada de instituciones
    df_author2institution = df_author2institution_exploded[['author_id']].join(df_institution_norm)

    df_work2author['load_datetime'] = date.today()
    df_author2institution['load_datetime'] = date.today()

    return df_work2author, df_author2institution


## Ejecuto Nodo

In [7]:
df_work2author, df_author2institution = land_work2authorships_openalex(df_work_raw)

[02/01/25 19:40:27] WARNING  /tmp/ipykernel_23490/167911302.py:29: SettingWithCopyWarning:          ]8;id=383008;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=725129;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_work2author['load_datetime'] = date.today()                                      
                                                                                                                   

# Resultados

In [8]:
df_work2author

,work_id,author_id,author_position,load_datetime
0,https://openalex.org/W2089852843,https://openalex.org/A5051451815,first,2025-02-01
1,https://openalex.org/W2089852843,https://openalex.org/A5110175901,middle,2025-02-01
2,https://openalex.org/W2089852843,https://openalex.org/A5059654447,last,2025-02-01
3,https://openalex.org/W3037375387,https://openalex.org/A5069680875,first,2025-02-01
4,https://openalex.org/W3037375387,https://openalex.org/A5031160102,middle,2025-02-01
...,...,...,...,...
5245,https://openalex.org/W2738753887,https://openalex.org/A5037617315,middle,2025-02-01
5246,https://openalex.org/W2738753887,https://openalex.org/A5034990546,middle,2025-02-01
5247,https://openalex.org/W2738753887,https://openalex.org/A5043451966,middle,2025-02-01
5248,https://openalex.org/W2738753887,https://openalex.org/A5079671858,middle,2025-02-01


In [9]:
df_author2institution

,author_id,country_code,display_name,id,ror,type,load_datetime
0,https://openalex.org/A5051451815,BR,Centro Brasileiro de Pesquisas Físicas,https://openalex.org/I4210125245,https://ror.org/02wnmk332,facility,2025-02-01
1,https://openalex.org/A5110175901,BR,Universidade Estadual de Maringá,https://openalex.org/I123443094,https://ror.org/04bqqa360,education,2025-02-01
2,https://openalex.org/A5059654447,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government,2025-02-01
3,https://openalex.org/A5059654447,BR,Centro Brasileiro de Pesquisas Físicas,https://openalex.org/I4210125245,https://ror.org/02wnmk332,facility,2025-02-01
4,https://openalex.org/A5059654447,AR,Universidad Nacional de La Plata,https://openalex.org/I874386039,https://ror.org/01tjs6929,education,2025-02-01
...,...,...,...,...,...,...,...
9654,https://openalex.org/A5034990546,AR,Consejo Nacional de Investigaciones Científica...,https://openalex.org/I151201029,https://ror.org/03cqe8w59,government,2025-02-01
9655,https://openalex.org/A5043451966,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government,2025-02-01
9656,https://openalex.org/A5079671858,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government,2025-02-01
9657,https://openalex.org/A5018751170,AR,National University of Quilmes,https://openalex.org/I138504249,https://ror.org/01r53hz59,funder,2025-02-01
